In [1]:
import geopandas as gpd
import pandas as pd
from bs4 import BeautifulSoup
import folium
import regex as re
import requests
from shapely.geometry import Polygon


In [2]:
#List of Russian regions
regs=['Республика Адыгея (Адыгея)','Алтайский край','Амурская обл.',
      'Архангельская обл.','Астраханская обл.','Республика Башкортостан',
      'Белгородская обл.','Брянская обл.','Республика Бурятия',
     'Чеченская Республика','Челябинская обл.','Чукотский АО','Чувашская Республика - Чувашия','г. Санкт-Петербург',
      'Республика Дагестан','Республика Алтай','Республика Ингушетия','Иркутская обл.','Ивановская обл.',
      'Кабардино-Балкарская Республика','Калининградская обл.','Республика Калмыкия','Калужская обл.','Камчатский край',
      'Карачаево-Черкесская Республика','Республика Карелия','Кемеровская обл. - Кузбасс','Хабаровский край','Республика Хакасия',
      'Ханты-Мансийский АО - Югра','Кировская обл.','Республика Коми','Костромская обл.','Краснодарский край',
      'Красноярский край','Курганская обл.','Курская обл.','Ленинградская обл.','Липецкая обл.','Магаданская обл.','Республика Марий Эл',
     'Республика Мордовия','г. Москва','Московская обл.','Мурманская обл.','Ненецкий АО','Нижегородская обл.','Республика Северная Осетия - Алания',
     'Новгородская обл.','Новосибирская обл.','Омская обл.','Орловская обл.','Оренбургская обл.','Пензенская обл.','Пермский край',
     'Приморский край','Псковская обл.','Ростовская обл.','Рязанская обл.','Республика Саха (Якутия)','Сахалинская обл.','Самарская обл.',
      'Саратовская обл.','Смоленская обл.','Ставропольский край','Свердловская обл.','Тамбовская обл.','Республика Татарстан (Татарстан)',
     'Томская обл.','Тульская обл.','Республика Тыва','Тверская обл.','Тюменская обл.','Удмуртская Республика','Ульяновская обл.',
     'Владимирская обл.','Волгоградская обл.','Вологодская обл.','Воронежская обл.','Ямало-Ненецкий АО','Ярославская обл.','Еврейская АО',
     'Забайкальский край','Республика Крым','город федерального значения Севастополь']


In [3]:
#However,there are no Crimea and Sevastopol in shape file .Importing .shp and parsing forgotten regions from OSM.
rr=gpd.read_file('Regions/gadm36_RUS_1.shp')
forgotten=['Республика Крым','Севастополь']
forgotten_coord=[]
for region in forgotten:
    d=requests.get('http://nominatim.openstreetmap.org/search?format=json&state='+region+'&polygon_geojson=1')
    forgotten_coord.append(Polygon(d.json()[0]['geojson']['coordinates'][0]))


In [4]:
#Append regions to geoseries and create geodataframe
rr=rr['geometry'].append(gpd.GeoSeries(forgotten_coord),ignore_index=True)
rr=gpd.GeoDataFrame(rr,crs={'init': 'epsg:3857'})
rr.columns=['geometry']
rr['region']=regs
rr=rr[['region','geometry']].sort_values('region')
rr.index=[i for i in range(len(rr.index))]

<ipython-input-4-a12bdac3f2bc>:3: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  rr=gpd.GeoDataFrame(rr,crs={'init': 'epsg:3857'})


In [5]:
#Get table with statistics
g=requests.get('http://fincan.ru/articles/117_dotacii-regionam-rossii-2020/')
soup=BeautifulSoup(g.text,'lxml')
table_rows=soup.find_all('table')[0].find_all('tr')
table_rows=table_rows[1:86]

#Get region names and subventions. 
regions=[]
subvention=[]
for row in table_rows:
    reg,mon=row.find_all('td')
    regions.append(str(reg).replace('<td>','').replace('</td>',''))
    subvention.append(str(mon).replace('<td>','').replace('</td>',''))


In [6]:
#Made Pandas DataFrame and convert subventions to float
rd=pd.DataFrame([regions,subvention]).T
rd.columns=['region','subvention']
rd['subvention']=rd.subvention.str.replace(',','.').astype('float')
rd.sort_values(by='region',inplace=True,ignore_index=True)

In [7]:
#Parsing table with region's incomes
raw_data=requests.get('http://bujet.ru/article/396309.php')
soup = BeautifulSoup(raw_data.text, 'lxml')
tables=soup.find_all('table')[0]
raw_data=tables.find_all('tr')[2:88]


In [8]:
regions1=[]
budgets=[]
for row in raw_data:
    regions1.append(re.findall(r'\S[A-Яа-я- ]+',row.find_all('td')[1].text)[0])
    budgets.append(re.findall(r'\S[0-9,]+',row.find_all('td')[2].text)[0].replace(',','.'))

In [9]:
reg_inc=pd.DataFrame([regions1,budgets]).T
reg_inc.columns=['region','income']
reg_inc.sort_values('region',ignore_index=True,inplace=True)
rr.iloc[[82,83,84],0]=rr.iloc[[82,83,84],0].str.replace('город федерального значения ','').str.replace('г. ','')
rr.sort_values('region',inplace=True,ignore_index=True)
rd.iloc[[82,83,84],0]=rd.iloc[[82,83,84],0].str.replace('город федерального значения ','').str.replace('г. ','')
rd.sort_values('region',inplace=True,ignore_index=True)
rd['income']=reg_inc['income']
rd['%']=round(rd['subvention']/rd['income'].astype('float')*100,2)
m=folium.Map()
rr=rr.merge(rd,on='region')
rd=rd.query('subvention!=0')
rr=rr.query('subvention!=0')

C:\Users\timna\anaconda3\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [10]:
#It is time for folium!
g=folium.Choropleth(geo_data=rr[['region','geometry','%']].to_json(),
    data=rd,
    name='choropleth',
    columns=['region','%'],
    key_on='feature.properties.region',
    fill_color='Spectral',
    fill_opacity=0.5,
    line_opacity=0.5,
    legend_name='percentage of subsidies',
    highlight = True).add_to(m)
g.geojson.add_child(
    folium.features.GeoJsonTooltip(['region','%']))
m.save('reg_subs.html')

In [11]:
rr.to_file('Regions/Rus_Reg.geojson',encoding='utf-8',driver='GeoJSON')